In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.linear_model import Lasso,LassoLarsCV, LassoLars, ElasticNetCV, ElasticNet
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
data = pd.read_csv("PGT121_Neu_OccAA.csv", header =0)

In [3]:
Y = data.values[:, 1]
features = data.values[:, 5:]
labels = data.columns.values[5:]
# features = features[:, lasso_inds]
filter_mask = np.where(np.all(features == features[0,:], axis = 0))
X = np.delete(features, filter_mask, axis = 1)
labels = np.delete(labels, filter_mask)
names = data.values[:, 3]
X = X.astype("float")
Y = Y.astype("float")

In [4]:
import time
from collections import Counter

In [5]:
r2s = []
mse = []
t0 = time.time()
best_params = []
param_counts = {
    "C":Counter(),
    "gamma":Counter(),
    "epsilon":Counter(),
    'degree':Counter(),
    'kernel':Counter()
}
# for i in range(3):
scores = ['r2']
# print("Running test {} of 3".format(i+1))
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.9)
# X_train, X_test, y_train, y_test = (X, X, Y, Y)
t1 = time.time()
for score in scores:
    t3 = time.time()
    params = [
        {'degree':[2, 3, 4, 5, 6], 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 1e5, 1e6, 1e7, 1e8, 1e9],  'gamma': [10, 1.0, 0.1, 0.01, 0.001, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9], 'kernel':['poly', 'rbf']}
    ]
    print("Tuning parameters for {}\n".format(score), flush = True)
    reg = GridSearchCV(SVR(), params, cv=10, scoring = score, n_jobs = 8, verbose = 3)
    reg.fit(X, Y)
    best_params.append("Best parameters: {}".format(reg.best_params_))
#     param_counts['C'][reg.best_params_["C"]]+=1
#     param_counts['gamma'][reg.best_params_["gamma"]]+=1
#     param_counts['gamma'][reg.best_params_["gamma"]]+=1
#     param_counts['gamma'][reg.best_params_["gamma"]]+=1
#     param_counts['gamma'][reg.best_params_["gamma"]]+=1
    print("Best parameters:\n\n{}".format(reg.best_params_), flush = True)
    means = reg.cv_results_['mean_test_score']
    stds = reg.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, reg.cv_results_['params']):
        print("%0.4f (+/-%0.04f) for %r" % (mean, std * 2, params), flush = True)

    y_pred = reg.predict(X_test)
    print("r2: {}".format(r2_score(y_test, y_pred)))
    print("MSE: {}".format(mean_squared_error(y_test, y_pred)))
    t4 = time.time()
    print("%0.2f minutes elapsed" %((t4-t3)/60), flush = True)
    r2s.append(r2_score(y_test, y_pred))
    mse.append(mean_squared_error(y_test, y_pred))
    
t2 = time.time()
print("%0.2f minutes elapsed" %((t2-t1)/60), flush = True)
    
print(r2s)
print(mse)
print(Counter(best_params).most_common())
print(param_counts)

Tuning parameters for r2

Fitting 10 folds for each of 1430 candidates, totalling 14300 fits


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   16.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:  3.4min
[Parallel(n_jobs=8)]: Done 496 tasks      | elapsed:  6.0min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  9.4min
[Parallel(n_jobs=8)]: Done 1136 tasks      | elapsed: 13.5min
[Parallel(n_jobs=8)]: Done 1552 tasks      | elapsed: 18.4min
[Parallel(n_jobs=8)]: Done 2032 tasks      | elapsed: 24.0min
[Parallel(n_jobs=8)]: Done 2576 tasks      | elapsed: 30.3min
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed: 37.3min
[Parallel(n_jobs=8)]: Done 3856 tasks      | elapsed: 45.0min
[Parallel(n_jobs=8)]: Done 4592 tasks      | elapsed: 450.8min
[Parallel(n_jobs=8)]: Done 5392 tasks      | elapsed: 460.0min
[Parallel(n_jobs=8)]: Done 6256 tasks      | elapsed: 469.9min
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed: 480.6min
[Parallel(n_jobs=8)]: Done 8176 tasks      | elapsed: 492.0min
[Paralle

Best parameters:

{'C': 0.001, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.1080 (+/-0.8176) for {'C': 0.0001, 'degree': 2, 'gamma': 10, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 10, 'kernel': 'rbf'}
0.1379 (+/-0.8257) for {'C': 0.0001, 'degree': 2, 'gamma': 1.0, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 1.0, 'kernel': 'rbf'}
-0.3823 (+/-0.3096) for {'C': 0.0001, 'degree': 2, 'gamma': 0.1, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
-0.4674 (+/-0.3056) for {'C': 0.0001, 'degree': 2, 'gamma': 0.01, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 0.001, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 2, 'gamma': 0.0001, 'kernel': 'poly'}
-0.4686 (+

-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 0.0001, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 0.0001, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-05, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-05, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-06, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-06, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-07, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-07, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-08, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-08, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-09, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.0001, 'degree': 6, 'gamma': 1e-09, '

-0.4686 (+/-0.3077) for {'C': 0.001, 'degree': 5, 'gamma': 1e-09, 'kernel': 'rbf'}
0.1936 (+/-0.6670) for {'C': 0.001, 'degree': 6, 'gamma': 10, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.001, 'degree': 6, 'gamma': 10, 'kernel': 'rbf'}
0.1936 (+/-0.6670) for {'C': 0.001, 'degree': 6, 'gamma': 1.0, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.001, 'degree': 6, 'gamma': 1.0, 'kernel': 'rbf'}
0.1936 (+/-0.6670) for {'C': 0.001, 'degree': 6, 'gamma': 0.1, 'kernel': 'poly'}
-0.4686 (+/-0.3077) for {'C': 0.001, 'degree': 6, 'gamma': 0.1, 'kernel': 'rbf'}
0.1959 (+/-0.6669) for {'C': 0.001, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
-0.4686 (+/-0.3076) for {'C': 0.001, 'degree': 6, 'gamma': 0.01, 'kernel': 'rbf'}
-0.4686 (+/-0.3076) for {'C': 0.001, 'degree': 6, 'gamma': 0.001, 'kernel': 'poly'}
-0.4685 (+/-0.3075) for {'C': 0.001, 'degree': 6, 'gamma': 0.001, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 0.001, 'degree': 6, 'gamma': 0.0001, 'kernel': 'poly'}
-0.4686 (+/-0.307

...

0.0751 (+/-0.8754) for {'C': 1000000.0, 'degree': 2, 'gamma': 1e-07, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 1000000.0, 'degree': 2, 'gamma': 1e-08, 'kernel': 'poly'}
-0.0612 (+/-0.4692) for {'C': 1000000.0, 'degree': 2, 'gamma': 1e-08, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 1000000.0, 'degree': 2, 'gamma': 1e-09, 'kernel': 'poly'}
-0.3429 (+/-0.3115) for {'C': 1000000.0, 'degree': 2, 'gamma': 1e-09, 'kernel': 'rbf'}
0.1523 (+/-0.7856) for {'C': 1000000.0, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
-0.0484 (+/-0.1266) for {'C': 1000000.0, 'degree': 3, 'gamma': 10, 'kernel': 'rbf'}
0.1523 (+/-0.7856) for {'C': 1000000.0, 'degree': 3, 'gamma': 1.0, 'kernel': 'poly'}
-0.0467 (+/-0.1246) for {'C': 1000000.0, 'degree': 3, 'gamma': 1.0, 'kernel': 'rbf'}
0.1523 (+/-0.7856) for {'C': 1000000.0, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}
-0.0393 (+/-0.1215) for {'C': 1000000.0, 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf'}
0.1523 (+/-0.7856) for {'C': 1000000.0, 'degree': 3, 'ga

0.1080 (+/-0.8176) for {'C': 10000000.0, 'degree': 2, 'gamma': 1.0, 'kernel': 'poly'}
-0.0467 (+/-0.1246) for {'C': 10000000.0, 'degree': 2, 'gamma': 1.0, 'kernel': 'rbf'}
0.1080 (+/-0.8176) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.1, 'kernel': 'poly'}
-0.0393 (+/-0.1215) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
0.1080 (+/-0.8176) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.01, 'kernel': 'poly'}
0.1365 (+/-0.3678) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
0.1080 (+/-0.8176) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'poly'}
0.1323 (+/-0.8068) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf'}
0.1080 (+/-0.8176) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.0001, 'kernel': 'poly'}
0.0197 (+/-0.8234) for {'C': 10000000.0, 'degree': 2, 'gamma': 0.0001, 'kernel': 'rbf'}
0.1192 (+/-0.8227) for {'C': 10000000.0, 'degree': 2, 'gamma': 1e-05, 'kernel': 'poly'}
-0.0042 (+/-0.8196) for {'C': 10000000.0, '

0.1936 (+/-0.6670) for {'C': 10000000.0, 'degree': 6, 'gamma': 0.001, 'kernel': 'poly'}
0.1323 (+/-0.8068) for {'C': 10000000.0, 'degree': 6, 'gamma': 0.001, 'kernel': 'rbf'}
-0.1120 (+/-0.3853) for {'C': 10000000.0, 'degree': 6, 'gamma': 0.0001, 'kernel': 'poly'}
0.0197 (+/-0.8234) for {'C': 10000000.0, 'degree': 6, 'gamma': 0.0001, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-05, 'kernel': 'poly'}
-0.0042 (+/-0.8196) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-05, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-06, 'kernel': 'poly'}
-0.0071 (+/-0.8231) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-06, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-07, 'kernel': 'poly'}
0.0097 (+/-0.8161) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-07, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-08, 'kernel': 'poly'}
0.0432 (+/-0.9600) for {'C':

-0.4678 (+/-0.3062) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-05, 'kernel': 'poly'}
-0.0042 (+/-0.8196) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-05, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-06, 'kernel': 'poly'}
-0.0071 (+/-0.8231) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-06, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-07, 'kernel': 'poly'}
-0.0071 (+/-0.7677) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-07, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-08, 'kernel': 'poly'}
-0.4500 (+/-0.8839) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-08, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-09, 'kernel': 'poly'}
-1.0436 (+/-1.7701) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-09, 'kernel': 'rbf'}
0.1936 (+/-0.6670) for {'C': 100000000.0, 'degree': 6, 'gamma': 10, 'kernel': 'poly'}
-0.0484 (+/-0.1266)

-0.4686 (+/-0.3077) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-07, 'kernel': 'poly'}
-0.0071 (+/-0.7677) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-07, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-08, 'kernel': 'poly'}
-38.7963 (+/-30.8940) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-08, 'kernel': 'rbf'}
-0.4686 (+/-0.3077) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-09, 'kernel': 'poly'}
-163.4963 (+/-173.5096) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-09, 'kernel': 'rbf'}
0.1871 (+/-0.7072) for {'C': 1000000000.0, 'degree': 5, 'gamma': 10, 'kernel': 'poly'}
-0.0484 (+/-0.1266) for {'C': 1000000000.0, 'degree': 5, 'gamma': 10, 'kernel': 'rbf'}
0.1872 (+/-0.7072) for {'C': 1000000000.0, 'degree': 5, 'gamma': 1.0, 'kernel': 'poly'}
-0.0467 (+/-0.1246) for {'C': 1000000000.0, 'degree': 5, 'gamma': 1.0, 'kernel': 'rbf'}
0.1872 (+/-0.7072) for {'C': 1000000000.0, 'degree': 5, 'gamma': 0.1, 'kernel': 'poly'}
-0.0393 (+/-0

`{'epsilon':[0.001, 0.1, 0.01, 1, 10, 100, 1000],'degree':[2, 3, 4, 5, 6], 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],  'gamma': [10, 1.0, 0.1, 0.01, 0.001], 'kernel':['poly']}`

= Best parameters: `{'C': 0.0001, 'degree': 4, 'epsilon': 0.001, 'gamma': 1.0, 'kernel': 'poly'}`

r2: 0.21569766913456467

MSE: 300.6242320691153

`{'degree':[2, 3, 4, 5, 6], 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 1e5, 1e6, 1e7, 1e8, 1e9],  'gamma': [10, 1.0, 0.1, 0.01, 0.001, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9], 'kernel':['poly', 'rbf']},`

`Best parameters: {'C': 1000000.0, 'degree': 4, 'gamma': 0.0001, 'kernel': 'poly'}`

[0.3276854867597826]
[245.07934960110524]

In [6]:
print(r2s)
print(mse)
print(Counter(best_params).most_common())
print(param_counts)

[0.99997750984606715]
[0.0099944872809808773]
[("Best parameters: {'C': 0.001, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}", 1)]
{'C': Counter(), 'gamma': Counter(), 'epsilon': Counter(), 'degree': Counter(), 'kernel': Counter()}


In [7]:
sort_means = means
sort_stds = stds
sort_params = reg.cv_results_['params']

In [8]:
sort_means, sort_stds, sort_params = zip(*sorted(zip(sort_means, sort_stds, sort_params), key = lambda x:x[0], reverse = True))
print(len(sort_params))
print(reg.cv_results_['params'][1])

1430
{'C': 0.0001, 'degree': 2, 'gamma': 10, 'kernel': 'rbf'}


In [9]:
for mean, std, params in zip(sort_means, sort_stds, sort_params):
        print("%0.20f (+/-%0.20f) for %r" % (mean, std * 2, params), flush = True)

0.19592167850831451736 (+/-0.66687928698502041414) for {'C': 0.001, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.19592117053963414852 (+/-0.66688035543518775494) for {'C': 1000000000.0, 'degree': 6, 'gamma': 0.0001, 'kernel': 'poly'}
0.19592073101264972923 (+/-0.66687977551027199929) for {'C': 1000, 'degree': 6, 'gamma': 0.001, 'kernel': 'poly'}
0.19359575627263178599 (+/-0.66701166013240231756) for {'C': 0.01, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.19359575627263178599 (+/-0.66701166013240231756) for {'C': 0.1, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.19359575627263178599 (+/-0.66701166013240231756) for {'C': 1, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.19359575627263178599 (+/-0.66701166013240231756) for {'C': 10, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.19359575627263178599 (+/-0.66701166013240231756) for {'C': 100, 'degree': 6, 'gamma': 0.01, 'kernel': 'poly'}
0.19359575627263178599 (+/-0.66701166013240231756) for {'C': 1000, 'degree': 6, 'gamma': 0.

0.18715126875145327023 (+/-0.70723919610376217992) for {'C': 100000.0, 'degree': 5, 'gamma': 0.01, 'kernel': 'poly'}
0.18715126875145327023 (+/-0.70723919610376217992) for {'C': 1000000.0, 'degree': 5, 'gamma': 0.01, 'kernel': 'poly'}
0.18715126875145327023 (+/-0.70723919610376217992) for {'C': 10000000.0, 'degree': 5, 'gamma': 0.01, 'kernel': 'poly'}
0.18715126875145327023 (+/-0.70723919610376217992) for {'C': 100000000.0, 'degree': 5, 'gamma': 0.01, 'kernel': 'poly'}
0.18715126875145327023 (+/-0.70723919610376217992) for {'C': 1000000000.0, 'degree': 5, 'gamma': 0.01, 'kernel': 'poly'}
0.18715076954249448837 (+/-0.70724294989406688572) for {'C': 100000.0, 'degree': 5, 'gamma': 0.001, 'kernel': 'poly'}
0.18715076954249448837 (+/-0.70724294989406688572) for {'C': 1000000.0, 'degree': 5, 'gamma': 0.001, 'kernel': 'poly'}
0.18715076954249448837 (+/-0.70724294989406688572) for {'C': 10000000.0, 'degree': 5, 'gamma': 0.001, 'kernel': 'poly'}
0.18715076954249448837 (+/-0.7072429498940668857

0.17487195265458954641 (+/-0.74715819771938118965) for {'C': 100000000.0, 'degree': 4, 'gamma': 0.001, 'kernel': 'poly'}
0.17487195265458954641 (+/-0.74715819771938118965) for {'C': 1000000000.0, 'degree': 4, 'gamma': 0.001, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 0.0001, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 0.001, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 0.01, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 0.1, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 1, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 10, 'degree': 4, 'gamma': 0.1, 'kernel': 'poly'}
0.17487146555103807999 (+/-0.74715712117380883583) for {'C': 100, 'degree': 4, 'gamma': 

0.15225301230077170622 (+/-0.78559466978061620601) for {'C': 100000.0, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
0.15225301230077170622 (+/-0.78559466978061620601) for {'C': 1000000.0, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
0.15225301230077170622 (+/-0.78559466978061620601) for {'C': 10000000.0, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
0.15225301230077170622 (+/-0.78559466978061620601) for {'C': 100000000.0, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
0.15225301230077170622 (+/-0.78559466978061620601) for {'C': 1000000000.0, 'degree': 3, 'gamma': 10, 'kernel': 'poly'}
0.15225210541705241907 (+/-0.78559810639343097272) for {'C': 10000000.0, 'degree': 3, 'gamma': 0.0001, 'kernel': 'poly'}
0.15225210541705241907 (+/-0.78559810639343097272) for {'C': 100000000.0, 'degree': 3, 'gamma': 0.0001, 'kernel': 'poly'}
0.15225210541705241907 (+/-0.78559810639343097272) for {'C': 1000000000.0, 'degree': 3, 'gamma': 0.0001, 'kernel': 'poly'}
0.15225155261876982538 (+/-0.78559861467164049387

0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 10000000.0, 'degree': 6, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 100000000.0, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 100000000.0, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 100000000.0, 'degree': 4, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 100000000.0, 'degree': 5, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 100000000.0, 'degree': 6, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 1000000000.0, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.36784120398570363530) for {'C': 1000000000.0, 'degree': 3, 'gamma': 0.01, 'kernel': 'rbf'}
0.13652090700018343350 (+/-0.367841203985703635

...

-0.46862822764508310636 (+/-0.30768142738156661897) for {'C': 0.001, 'degree': 3, 'gamma': 1e-09, 'kernel': 'rbf'}
-0.46862822764508310636 (+/-0.30768142738156661897) for {'C': 0.001, 'degree': 4, 'gamma': 1e-09, 'kernel': 'rbf'}
-0.46862822764508310636 (+/-0.30768142738156661897) for {'C': 0.001, 'degree': 5, 'gamma': 1e-09, 'kernel': 'rbf'}
-0.46862822764508310636 (+/-0.30768142738156661897) for {'C': 0.001, 'degree': 6, 'gamma': 1e-09, 'kernel': 'rbf'}
-0.46862822771636519770 (+/-0.30768142747556959105) for {'C': 0.001, 'degree': 2, 'gamma': 1e-06, 'kernel': 'poly'}
-0.46862822771636519770 (+/-0.30768142747556959105) for {'C': 0.1, 'degree': 2, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822771636519770 (+/-0.30768142747556959105) for {'C': 10, 'degree': 2, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822771636519770 (+/-0.30768142747556959105) for {'C': 1000, 'degree': 2, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822772106754732 (+/-0.30768142748219623472) for {'C': 1000000000.0, 'degre

-0.46862822784430613332 (+/-0.30768142767111544833) for {'C': 100000000.0, 'degree': 5, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822784432506262 (+/-0.30768142767114264879) for {'C': 1, 'degree': 6, 'gamma': 1e-05, 'kernel': 'poly'}
-0.46862822784432506262 (+/-0.30768142767114264879) for {'C': 1000000.0, 'degree': 6, 'gamma': 1e-06, 'kernel': 'poly'}
-0.46862822784438035173 (+/-0.30768142767123046744) for {'C': 0.001, 'degree': 2, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822784438035173 (+/-0.30768142767123046744) for {'C': 0.1, 'degree': 2, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784438079582 (+/-0.30768142767123141113) for {'C': 0.0001, 'degree': 3, 'gamma': 1e-06, 'kernel': 'poly'}
-0.46862822784438079582 (+/-0.30768142767123141113) for {'C': 0.1, 'degree': 3, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822784438079582 (+/-0.30768142767123141113) for {'C': 100, 'degree': 3, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822784438079582 (+/-0.30768142767123141113) for {'C': 100000.0,

-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 4, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 4, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 4, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 5, 'gamma': 1e-06, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 5, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 5, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 5, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 6, 'gamma': 1e-06, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 0.01, 'degree': 6,

-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 1000000.0, 'degree': 5, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 1000000.0, 'degree': 6, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 1000000.0, 'degree': 6, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 1000000.0, 'degree': 6, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 10000000.0, 'degree': 4, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 10000000.0, 'degree': 5, 'gamma': 1e-08, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 10000000.0, 'degree': 5, 'gamma': 1e-09, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.30768142767124995185) for {'C': 10000000.0, 'degree': 6, 'gamma': 1e-07, 'kernel': 'poly'}
-0.46862822784439323032 (+/-0.307681

-38.79628724518832427748 (+/-30.89402050594477699974) for {'C': 1000000000.0, 'degree': 3, 'gamma': 1e-08, 'kernel': 'rbf'}
-38.79628724518832427748 (+/-30.89402050594477699974) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-08, 'kernel': 'rbf'}
-38.79628724518832427748 (+/-30.89402050594477699974) for {'C': 1000000000.0, 'degree': 5, 'gamma': 1e-08, 'kernel': 'rbf'}
-38.79628724518832427748 (+/-30.89402050594477699974) for {'C': 1000000000.0, 'degree': 6, 'gamma': 1e-08, 'kernel': 'rbf'}
-163.49627838158991721684 (+/-173.50964280705119335835) for {'C': 1000000000.0, 'degree': 2, 'gamma': 1e-09, 'kernel': 'rbf'}
-163.49627838158991721684 (+/-173.50964280705119335835) for {'C': 1000000000.0, 'degree': 3, 'gamma': 1e-09, 'kernel': 'rbf'}
-163.49627838158991721684 (+/-173.50964280705119335835) for {'C': 1000000000.0, 'degree': 4, 'gamma': 1e-09, 'kernel': 'rbf'}
-163.49627838158991721684 (+/-173.50964280705119335835) for {'C': 1000000000.0, 'degree': 5, 'gamma': 1e-09, 'kernel': 'rbf'}


In [ ]:
#{'degree': 4, 'C': 0.1, 'kernel': 'poly', 'gamma': 0.01} - 53 min (MSE) r^2 ~ .25
#{'degree': 4, 'C': 0.1, 'kernel': 'poly', 'gamma': 0.01} - 52.1 min (R2) r^2 ~ 0.251, neg_MSE = 273.8


#{'degree': 5, 'C': 0.01, 'kernel': 'poly', 'gamma': 0.01} - 6.3 min (MSE) r^2 ~0.254, neg_MSE = 272.5
#{'degree': 5, 'C': 0.01, 'kernel': 'poly', 'gamma': 0.01} - 5.0 min (r2) r^2 ~~0.254, neg_MSE = 272.5

#{'kernel': 'poly', 'C': 0.01, 'gamma': 0.01, 'degree': 5} - 3.5 min (r2) r2 ~~0.30, neg_MSE = 252.1
#{'kernel': 'poly', 'C': 0.005, 'gamma': 0.01, 'degree': 5} - r2 ~~ 0.3, neg_MSE = 250
#{'degree': 5, 'C': 0.0025, 'gamma': 0.01, 'kernel': 'poly'} - r2 ~~ 0.39, neg_MSE = 230

#{'C': 0.0025, 'degree': 5, 'gamma': 0.009, 'kernel': 'poly'} - r2 = 0.412

#C:0.0025, gamma:6, degree :5, kernel: poly